# <b>DS3000 Group Project</b>

# <b>Chef.AI</b> - Lukas Bozinov & Arjun Atwal - Group 31

## Section 0 - Preliminary Code

### Section 0.1: Import all necessary libraries

In [13]:
import numpy as np #etc etc etc

### Section 0.2: Other things

In [14]:
print('balls in 3D')

balls in 3D


## Section 1 - Organization of Dataset

### Section 1.1: Parse the dataset

In [15]:
print('balls')

balls


### Section 1.2: Data Pre-Processing

In [16]:
print('2+2=5')

2+2=5


## Section 2 - Mathematical Function Initialization

### Section 2.1: Sigmoid Function Declaration

In [17]:
print('sigma')

sigma


### Section 2.2: Other Mathematical Functions

In [18]:
print('more math')

more math


## Section 3 - Modelling

### Section 3.1: Load data into model

In [19]:
print('LOAD IT MAN')

LOAD IT MAN


### Section 3.2: Fit model

In [20]:
print('model.fit go brrrrr')

model.fit go brrrrr


### Section 3.3: Evaluate accuracy

In [21]:
print('auroc man idk')

auroc man idk


## Section 4 - Visualization

### Section 4.1: Line graph

In [22]:
print('something something matplotlib')

something something matplotlib


### Section 4.2: Other visualization

In [ ]:
print('something something seaborn')

## Section 5 - Comparisons to other models (?)

### Section 5.1: Comparison to linear regression

In [ ]:
print('news flash: linear regression sucks')

## Section 6 - Conclusions to draw

In [ ]:
print('overall, our model is better than the other one')